# Set up


In [ ]:
!git clone https://github.com/Baker-Data-Science/ergm-ensemble-embedding

Cloning into 'ergm-ensemble-embedding'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (21/21), done.
Receiving objects: 100% (35/35), 37.84 KiB | 2.70 MiB/s, done.
remote: Total 35 (delta 16), reused 32 (delta 13), pack-reused 0 (from 0)
Resolving deltas: 100% (16/16), done.


In [ ]:
%%capture
%cd /content/
!git clone https://github.com/liufule12/repDNA.git
%cd /content/repDNA/repDNA

from nac import Kmer, get_kmer_list

In [ ]:
%%capture
%cd /content/
!rm -rf /content/Aptamers
!git clone https://github.com/Baker-Data-Science/GMFold.git /content/Aptamers
%cd /content/Aptamers/src
from GMfold import gmfold, gm_s_matrix, gm_dot_bracket

In [ ]:
!pip3 install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [ ]:
!pip install viennarna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 46.2 MB/s eta 0:00:00


In [ ]:
!pip install ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.6 MB/s eta 0:00:00


In [ ]:
# imports
import ast, re
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.cluster import SpectralClustering
from sklearn.decomposition import NMF
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from collections import defaultdict
import math


In [ ]:
%cd /content/ergm-ensemble-embedding
from featurize import reconstruct_structs_from_dotbracket
from featurize import compute_structure_probabilities, normalize_probs
%cd /content/

/content/ergm-ensemble-embedding
/content


In [ ]:
# SET PLOT PARAMETERS
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.titlesize'] = 40
plt.rcParams['xtick.labelsize'] = 30 # 40
plt.rcParams['ytick.labelsize'] = 30# 40
plt.rcParams['legend.fontsize'] = 40
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [ ]:
# Featureized Data Files
!aws s3 cp s3://ucla-ds/data/fold-data/df_9th.csv ./data/ --no-sign-request
!aws s3 cp s3://ucla-ds/data/fold-data/df_12th.csv ./data/ --no-sign-request
!aws s3 cp s3://ucla-ds/data/fold-data/df_13th.csv ./data/ --no-sign-request
!aws s3 cp s3://ucla-ds/data/fold-data/df_16th.csv ./data/ --no-sign-request

download: s3://ucla-ds/data/fold-data/df_9th.csv to data/df_9th.csv 
download: s3://ucla-ds/data/fold-data/df_12th.csv to data/df_12th.csv
download: s3://ucla-ds/data/fold-data/df_13th.csv to data/df_13th.csv
download: s3://ucla-ds/data/fold-data/df_16th.csv to data/df_16th.csv


###Four rounds of SELEX data

In [ ]:
%cd /content/
df_9th = pd.read_csv("/content/Aptamers/data/published_clean_files/Published_Preprocessed_N48 after 9th.csv")
df_12th = pd.read_csv("/content/Aptamers/data/published_clean_files/Published_Preprocessed_N58 after 12th.csv")
df_13th = pd.read_csv("/content/Aptamers/data/published_clean_files/Published_Preprocessed_N48 after 13th.csv")
df_16th = pd.read_csv("/content/Aptamers/data/published_clean_files/Published_Preprocessed_N58 after 16th.csv")
df_clean = pd.read_csv("/content/Aptamers/data/fold_published.csv")

/content


In [ ]:
# Condensing library data into dataset of 4450 aptamers
df = pd.read_csv(r'/content/Aptamers/data/fold_published.csv')
df['faces'] = df['faces'].apply(ast.literal_eval)
df['energy_faces'] = df['energy_faces'].apply(ast.literal_eval)
db = df[['d_b']]
db = db.applymap(lambda x: list(x))
db = db.applymap(lambda x: [1 if y == '(' else -1 if y == ')' else 0 for y in x])
db = db.applymap(lambda x: np.cumsum(x))
df['motzkin_path'] = db

/tmp/ipython-input-426446042.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  db = db.applymap(lambda x: list(x))
/tmp/ipython-input-426446042.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  db = db.applymap(lambda x: [1 if y == '(' else -1 if y == ')' else 0 for y in x])
/tmp/ipython-input-426446042.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  db = db.applymap(lambda x: np.cumsum(x))


##Table for keeping track of counts and selective pressure

In [ ]:
df_clean_sequences = df['Sequence'].tolist()
table2 = pd.DataFrame({'Sequence': df_clean_sequences})

for df, label in zip(
    [df_9th, df_12th, df_13th, df_16th, df],
    ['R9', 'R12', 'R13', 'R16', 'Raw']
):
    col_name = f'Count_{label}'
    table2 = table2.merge(
        df[['Sequence', 'Count']].rename(columns={'Count': col_name}),
        on='Sequence',
        how='left'
    )

# print the length of table 2 filtered by 46 length sequences
table2['Count_Clean'] = table2[['Count_R13', 'Count_R16']].max(axis=1)

# 4) Fill missing counts with 0 and convert to integer
count_cols = [c for c in table2.columns if c.startswith('Count_')]
table2[count_cols] = table2[count_cols].fillna(0).astype(int)

n_bins = 10
# Define redistributing function
def redistribute_uniform(group):
    n = len(group)
    return np.linspace(0, 1, n, endpoint=False) + (0.5 / n)
def redistribute_to_quantile_range(group, q_bin, total_bins):
    # separate q_bin into the right bin integer
    q_bin = int(q_bin.split('Q')[1])
    n = len(group)
    rel_pos = np.linspace(0, 1, n, endpoint=False) + (0.5 / n)  # [0,1) within group
    start = q_bin / total_bins
    width = 1.0 / total_bins
    return start + rel_pos[::-1] * width  # map into bin's range

# Start from fresh table
dataframe = table2.copy()
groups = []

# Step 1: Bin counts 1–5 together
group_low = dataframe[dataframe['Count_Clean'].between(0, 5)].copy()
group_low['Bin_Label'] = -1
# group_low['Redist_Score'] = redistribute_uniform(group_low['Count_Clean'])
groups.append(group_low)

# Step 2: Bin remaining counts > 5 into percentiles
df_rest = dataframe[dataframe['Count_Clean'] > 5].copy()
df_rest['Bin_Label'] = pd.qcut(df_rest['Count_Clean'], q=n_bins-1, labels=False)
df_rest = pd.concat(groups + [df_rest], axis=0).sort_index()
df_rest['Bin_Label'] = 'Q' + (df_rest['Bin_Label']+1).astype(str)
# df_rest['Redist_Score'] = df_rest.groupby('Bin_Label')['Count_Clean'].transform(redistribute_uniform)
df_rest['Redist_Count'] = (
    df_rest.groupby('Bin_Label', group_keys=False)
    .apply(lambda g: pd.Series(redistribute_to_quantile_range(g, g['Bin_Label'].iloc[0], n_bins),
           index = g.index))
)
table2 = df_rest.copy()

# make a selective presure N48 column by assessing the proportion of increse or decrease in value
table2['N48_presure'] = table2['Count_R13'] - table2['Count_R9']
table2['N48_presure'] = table2['N48_presure'] / (table2['Count_R9'] + 1e-7)
# table2.loc[table2['Count_R9'] == 0, 'N48_presure'] = 0

table2['N58_presure'] = table2['Count_R16'] - table2['Count_R12']
table2['N58_presure'] = table2['N58_presure'] / (table2['Count_R12'] + 1e-7)
# table2.loc[table2['Count_R12'] == 0, 'N58_presure'] = 0

table2['N48_sign'] = np.sign(table2['N48_presure'])
table2['N58_sign'] = np.sign(table2['N58_presure'])
# map their sum
table2['Pressure_slope'] = table2['N48_presure'] + table2['N58_presure']
table2['Pressure_sign'] = table2['N48_sign'] + table2['N58_sign']
table2['Reliability'] = table2['Pressure_sign'] * table2['Redist_Count'] + (1 - table2['Pressure_sign']) * (1 - table2['Redist_Count'])
table2['Normalized_Count'] = table2['Count_Clean'] / table2['Count_Clean'].max()
table2['Reliability_2'] = table2['Pressure_sign'] * table2['Normalized_Count'] + (1 - table2['Pressure_sign']) * (1 - table2['Normalized_Count'])
table2['Log_Norm_Count'] = np.log10(table2['Count_Clean']+1e-10)
# now normalized to 0,1
table2['Log_Norm_Count'] = (table2['Log_Norm_Count'] - table2['Log_Norm_Count'].min()) / (table2['Log_Norm_Count'].max() - table2['Log_Norm_Count'].min())
# use log normalized for Reliability_3
table2['Reliability_3'] = table2['Pressure_sign'] * table2['Log_Norm_Count'] + (1 - table2['Pressure_sign']) * (1 - table2['Log_Norm_Count'])


# Noise gate
# if the count is 0 in round 9 and not in round 11 OR is 0 in round 12 and not in round 16 then set the noise gate to 0 otherwise set it to 1
# place holder
table2['Noise_Gate'] = 1
# if count is 0 in round 9 and not in round 11 # tra
table2.loc[((table2['Count_R9'] == 0) & (table2['Count_R13'] >= 0))
& (((table2['Count_R12'] == 0) & (table2['Count_R16'] >= 0)))
           , 'Noise_Gate'] = 0
table2.loc[table2['Noise_Gate'] == 0, 'Reliability_2'] = 0
table2.head()

/tmp/ipython-input-307237542.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(redistribute_to_quantile_range(g, g['Bin_Label'].iloc[0], n_bins),


,Sequence,Count_R9,Count_R12,Count_R13,Count_R16,Count_Raw,Count_Clean,Bin_Label,Redist_Count,N48_presure,...,N48_sign,N58_sign,Pressure_slope,Pressure_sign,Reliability,Normalized_Count,Reliability_2,Log_Norm_Count,Reliability_3,Noise_Gate
0,ACGACGGGGCACATTGTGCTGTTCATCTGTTCCGCAGGAGAGTCGT,55337,85,77352,556,77352,77352,Q9,0.998214,0.397835,...,1.0,1.0,5.939012,2.0,1.994643,1.000000,2.000000,1.000000,2.000000,1
1,ACGACCACCTAGGTATTCATGACCGTCTAGAGCTTTCATTGGTCGT,15383,52316,38755,67049,67049,67049,Q9,0.994643,1.519340,...,1.0,1.0,1.800955,2.0,1.983929,0.866804,1.600411,0.995830,1.987491,1
2,ACGACGGGGCACATTGTGCTATTCAGTTGTTCCGCAGGAGAGTCGT,3495,48708,622,8503,48708,8503,Q9,0.991071,-0.822031,...,-1.0,-1.0,-1.647461,-2.0,-1.955357,0.109926,2.450370,0.935595,-1.677973,1
3,ACGACGGGGCACATTGTGCTGTCCATCTGTTCCGCAGGAGAGTCGT,19164,1104,3488,28432,28432,28432,Q9,0.987500,-0.817992,...,-1.0,1.0,23.935631,0.0,0.012500,0.367566,0.632434,0.970805,0.029195,1
4,ACGACGGGGCACATTGTGCTGTTCACCTGTTCCGCAGGAGAGTCGT,12126,105,5947,2530,12126,5947,Q9,0.983929,-0.509566,...,-1.0,1.0,22.585672,0.0,0.016071,0.076882,0.923118,0.925165,0.074835,1


##Elimination of mistranscriptions

In [ ]:
transition_mask = (
    (
        (table2['Count_R9'] == 0) &
        (table2['Count_R13'] > 0) &
        (table2['Count_R12'] == 0) &
        (table2['Count_R16'] > 0)
    ) |
    (
        (table2['Count_R9'] == 0) &
        (table2['Count_R13'] > 0) &
        (table2['N58_presure'] < 0)
    ) |
    (
        (table2['Count_R9'] == 0) &
        (table2['Count_R13'] > 0) &
        (table2['Count_R12'] == 0) &
        (table2['Count_R16'] == 0)
    ) |
     (
        (table2['Count_R9'] == 0) &
        (table2['Count_R13'] == 0) &
        (table2['Count_R12'] == 0) &
        (table2['Count_R16'] > 0)
    )
)
print("Aptamers with 0→nonzero R9→R13 transitions:", transition_mask.sum())

Aptamers with 0→nonzero R9→R13 transitions: 739


In [ ]:
# Remove mistranscriptions
table2 = table2[~transition_mask].copy()

# Confirm new size
print(f"New table2 length after removing 0→nonzero R9→R13 transitions: {len(table2)}")

New table2 length after removing 0→nonzero R9→R13 transitions: 3711


##Embedding

### Calculate Boltzmann Distribution

In [ ]:
#----------Obtain the Boltzmann distribution for each primary sequence--------------------------

boltz_distrib = []

for seq in tqdm(table2['Sequence']):
  distribs = compute_structure_probabilities(seq)
  norm_distribs = normalize_probs(distribs) #renormalize probabilities since we only took top 50
  boltz_distrib.append(norm_distribs)

table2['Boltz_Distrib'] = boltz_distrib

100%|██████████| 3711/3711 [02:57<00:00, 20.91it/s]


In [ ]:
table2

,Sequence,Count_R9,Count_R12,Count_R13,Count_R16,Count_Raw,Count_Clean,Bin_Label,Redist_Count,N48_presure,...,N58_sign,Pressure_slope,Pressure_sign,Reliability,Normalized_Count,Reliability_2,Log_Norm_Count,Reliability_3,Noise_Gate,Boltz_Distrib
0,ACGACGGGGCACATTGTGCTGTTCATCTGTTCCGCAGGAGAGTCGT,55337,85,77352,556,77352,77352,Q9,0.998214,0.397835,...,1.0,5.939012,2.0,1.994643,1.000000,2.000000,1.000000,2.000000,1,[(((((((.(((((...))))).)((.(((((...))))).)))))...
1,ACGACCACCTAGGTATTCATGACCGTCTAGAGCTTTCATTGGTCGT,15383,52316,38755,67049,67049,67049,Q9,0.994643,1.519340,...,1.0,1.800955,2.0,1.983929,0.866804,1.600411,0.995830,1.987491,1,[((((((((...((((..((..((...))..))))))....)))))...
2,ACGACGGGGCACATTGTGCTATTCAGTTGTTCCGCAGGAGAGTCGT,3495,48708,622,8503,48708,8503,Q9,0.991071,-0.822031,...,-1.0,-1.647461,-2.0,-1.955357,0.109926,2.450370,0.935595,-1.677973,1,[(.(..(((((((.((((.......)))))))))))..)..........
3,ACGACGGGGCACATTGTGCTGTCCATCTGTTCCGCAGGAGAGTCGT,19164,1104,3488,28432,28432,28432,Q9,0.987500,-0.817992,...,1.0,23.935631,0.0,0.012500,0.367566,0.632434,0.970805,0.029195,1,[(..(((((.(((...))).)))))(.(((((...))))).).......
4,ACGACGGGGCACATTGTGCTGTTCACCTGTTCCGCAGGAGAGTCGT,12126,105,5947,2530,12126,5947,Q9,0.983929,-0.509566,...,1.0,22.585672,0.0,0.016071,0.076882,0.923118,0.925165,0.074835,1,[(((((((.(((((...))))).)((.(((((...))))).)))))...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4444,ACGACGCATCAGGCACGATTGCATGATCTGTCGT,1,0,0,0,1,0,Q0,0.000131,-1.000000,...,0.0,-1.000000,-1.0,1.999607,0.000000,2.000000,0.000000,2.000000,1,"[(((((((.((((.(((....))).)))).)))))), -12.6999..."
4445,ACGACGGGGCCACTATACCTTCAACCTCGATACGGGATGGTTCCGT...,1,0,0,0,1,0,Q0,0.000107,-1.000000,...,0.0,-1.000000,-1.0,1.999678,0.000000,2.000000,0.000000,2.000000,1,[((((((.((((.((...(((.((..((.......)))).)))......
4446,ACGACGGTCAGGCAACTGTAAAGTGTGCACTAGTCGT,1,0,0,0,1,0,Q0,0.000083,-1.000000,...,0.0,-1.000000,-1.0,1.999750,0.000000,2.000000,0.000000,2.000000,1,"[(((((((((...((((((....))).)))))).))))), -9.69..."
4447,ACGACGGCCTGATACCACCCAAGCACCTCTGGAATATCGGTGGTAA...,1,0,0,0,1,0,Q0,0.000060,-1.000000,...,0.0,-1.000000,-1.0,1.999821,0.000000,2.000000,0.000000,2.000000,1,[((((((...(((.(((((((......((...))......))))))...


###Build feature space

In [ ]:
from tqdm import tqdm

#------------Get bag of face distribution for each aptamer in the N48 library-------------------

face_energies_lib1 = []

# Filter aptamers with nonzero counts in both rounds
mask_lib1 = ~((table2['Count_R9'] == 0) & (table2['Count_R13'] == 0)) # If you calculated boltzmann distributions by hand, change table to table2
table_lib1 = table2[mask_lib1].reset_index(drop=True)

for idx, row in tqdm(table_lib1.iterrows(), total=len(table_lib1)):
    seq = row['Sequence']
    distrib = row['Boltz_Distrib']

    all_structs = []

    for (db, en, prob) in distrib:
        try:
            structs = reconstruct_structs_from_dotbracket(seq, db)
            struct_faces = [(s.desc, s.e) for s in structs if s.e != math.inf]
            all_structs.append(struct_faces)
        except Exception as e:
            print(f"Failed on idx {idx}, structure {db} → {e}")
            all_structs.append([])

    face_energies_lib1.append(all_structs)

In [ ]:
#-------Get feature space for N48 library based on bag of faces for each aptamer------
words_lib1 = []

# Iterate through each aptamer's list of lists of face-energy tuples
for struct_list in face_energies_lib1:  # list of structures
    for face_energy in struct_list:          # each structure: list of (face, energy)
        for face in face_energy:
          if face[0] not in words_lib1:
            words_lib1.append(face[0])  # add the descriptor

In [ ]:
#------------Get bag of face distribution for each aptamer in the N48 library-------------------

face_energies_lib2 = []

# Filter aptamers with nonzero counts in both rounds
mask_lib2 = ~((table2['Count_R12'] == 0) & (table2['Count_R16'] == 0))
table_lib2 = table2[mask_lib2].reset_index(drop=True)

for idx, row in tqdm(table_lib2.iterrows(), total=len(table_lib2)):
    seq = row['Sequence']
    distrib = row['Boltz_Distrib']  # list of (dotbracket, energy, prob)

    all_structs = []

    for (db, en, prob) in distrib:
        try:
            structs = reconstruct_structs_from_dotbracket(seq, db)
            struct_faces = [(s.desc, s.e) for s in structs if s.e != math.inf]
            all_structs.append(struct_faces)
        except Exception as e:
            print(f"Failed on idx {idx}, structure {db} → {e}")
            all_structs.append([])

    face_energies_lib2.append(all_structs)

In [ ]:
#-------Get feature space for N58 library based on bag of faces for each aptamer------
words_lib2 = []

# Iterate through each aptamer's list of lists of face-energy tuples
for struct_list in face_energies_lib2:  # list of structures
    for face_energy in struct_list:          # each structure: list of (face, energy)
        for face in face_energy:
          if face[0] not in words_lib2:
            words_lib2.append(face[0])  # add the descriptor

In [ ]:
#-------------kmer embedding generation---------------------
kmer = Kmer(k=4)
sequences = list(df['Sequence'].values)
K = np.array(kmer.make_kmer_vec(sequences))
kmer_list = get_kmer_list(kmer.k, kmer.upto, kmer.alphabet)

###Build expected bag of face matrix and feature space
Since table2 is used directly here, mistranscriptions are already filtered out

In [ ]:
face_embeddings = []

for i, row in table2.iterrows():
    faces_energy = row['face_energies']
    boltz_distrib = row['Boltz_Distrib']

    expected_bof = compute_expected_bag_of_faces(faces_energy, boltz_distrib, words0) #computes expected bag of faces for each aptamer
    face_embeddings.append(expected_bof)

# Save into a new column in table2
table2['Expected_BoFs'] = face_embeddings

In [ ]:
# Create expected bag of face matrix
X_Bof = np.vstack(table2['Expected_BoFs'].values)

In [ ]:
words_ordered = [w for w in words_lib1 if w in words_lib1 or w in words_lib2]
extra_words = [w for w in words_lib2 if w not in words_ordered] # Add remaining words from words_lib2 that aren't already included
words_combined = sorted(words_ordered + extra_words) #faces sorted alphabetically

In [ ]:
features = words + list(kmer_list)

In [ ]:
#----Make entire embedding matrix (kmer + expected BoF)----
bof_array = np.array(X_Bof.tolist())
kmer_array = np.array(K)

X_kBof = np.hstack([bof_array, kmer_array])

# Ridge Regressor

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from sklearn.linear_model import Ridge

In [ ]:
#-----Ridge regression model for feature analysis------------------------

# Filter aptamers that are present in either df9 or df13
counts_9 = table2['Count_R9']
counts_13 = table2['Count_R13']
mask_present = ~((counts_9 == 0) & (counts_13 == 0))  # Not zero in both

# Subset the X matrix and y values
X_lib1 = X_kBof[mask_present]
y_lib1 = table2['N48_presure'][mask_present].values.reshape(-1, 1)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_lib1_scaled = scaler_X.fit_transform(X_lib1)
y_lib1_scaled = scaler_y.fit_transform(y_lib1).ravel()

# Define Ridge regression + grid of alpha values
ridge = Ridge()
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
model = Ridge(alpha=100, solver='lsqr')  # Try alpha=0.1, 1.0, or 10.0
model.fit(X_lib1_scaled, y_lib1_scaled)

# Evaluation
y_pred_lib1 = model.predict(X_lib1_scaled)
r2_lib1 = r2_score(y_lib1_scaled, y_pred_lib1)
corr_lib1, _ = pearsonr(y_lib1_scaled, y_pred_lib1)

print(f"R² on Lib1 aptamers: {r2_lib1:.4f}")
print(f"Pearson ρ: {corr_lib1:.4f}")

In [ ]:
len(model.coef_)
len(features)

In [ ]:
# Sort by absolute value of coefficients
coef_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': model.coef_
})

coef_df_sorted = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)

# Filter negative coefficients and sort from most negative to least
negative_coef_df = coef_df[coef_df['Coefficient'] < 0].sort_values(by='Coefficient')
negative_features = negative_coef_df['Feature'].tolist()

In [ ]:
feature_names = list(features)  # Assuming 'features' contains BoF + kmer names
coefficients = model.coef_

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Sort by absolute value (optional)
coef_df_sorted = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)

# Display top 10 features
print("Top 10 features by absolute coefficient:")
print(coef_df_sorted.head(10))

In [ ]:
# Filter negative coefficients and sort from most negative to least
negative_coef_df = coef_df[coef_df['Coefficient'] < 0].sort_values(by='Coefficient')
negative_features = negative_coef_df['Feature'].tolist()

# Display
print("Features with Negative Coefficients (Most to Least Negative):")
print(negative_coef_df)

In [ ]:
kwords_lib2 = words_lib2 + kmer_list

In [ ]:
# Step 1: Get the overlap set (features must be in both lists)
overlap_set = set(negative_features) & set(kwords_lib2)

# Step 2: Filter rows in negative_coef_df for features in overlap_set
# and sort by coefficient (ascending → most negative to least negative)
sorted_overlap_df = negative_coef_df[
    negative_coef_df['Feature'].isin(overlap_set)
].sort_values(by='Coefficient')  # assumes column is named 'Coefficient'

# Step 3: Extract features in sorted order
overlap_negative_features = list(sorted_overlap_df['Feature'])

print(len(overlap_negative_features))

In [ ]:
# Filter positive coefficients and sort from most negative to least
positive_coef_df = coef_df[coef_df['Coefficient'] > 0].sort_values(by='Coefficient', ascending = False)
positive_features = positive_coef_df['Feature'].tolist()

# Display
print("Features with Posotive Coefficients (Biggest to Smallest):")
print(positive_coef_df[:10])

In [ ]:
# Step 1: Get the overlap set (features must be in both lists)
overlap_set = set(positive_features) & set(kwords_lib2)

# Step 2: Filter rows in negative_coef_df for features in overlap_set
# and sort by coefficient (ascending → most negative to least negative)
sorted_overlap_df = positive_coef_df[
    positive_coef_df['Feature'].isin(overlap_set)
].sort_values(by='Coefficient', ascending=False)  # assumes column is named 'Coefficient'

# Step 3: Extract features in sorted order
overlap_positive_features = list(sorted_overlap_df['Feature'])

print(len(overlap_positive_features))

##Anomalous clusters detection

In [ ]:
#Extract negative features in general (overlap or not)
top_neg_features = negative_coef_df['Feature'].tolist()
top_pos_features = positive_coef_df['Feature'].tolist()

# Create a mapping from feature name to its column index
feature_to_index = {name: idx for idx, name in enumerate(features)}

# Get column indices for the selected negative features
neg_feature_indices = [feature_to_index[name] for name in top_neg_features]

# Extract columns from X_kBof to form cluster input matrix C
C = X_kBof[:, neg_feature_indices]

In [ ]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

# Ensure all values in C are non-negative
# Use MinMaxScaler to scale each feature between 0 and 1
scaler_nmf = MinMaxScaler()
C_nonneg = scaler_nmf.fit_transform(C)

# Fit NMF
nmf_model = NMF(n_components=25, init='nndsvda', random_state=42, max_iter=1000)
W = nmf_model.fit_transform(C_nonneg)  # Aptamer-topic matrix (samples × components)
H = nmf_model.components_              # Feature-topic matrix (components × features)

# View shape of outputs
print("W (Aptamer-topic matrix) shape:", W.shape)
print("H (Component-feature matrix) shape:", H.shape)

# Spectral clustering
n_clusters = 25  # or choose based on silhouette score, etc.

# Run Spectral Clustering
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
kBof_clusters = spectral.fit_predict(W)

In [ ]:
#Choose top 10 features for average calculation using overlapping features as cluster embedding
top10_negative_face_features = overlap_negative_features[:5]
top10_positive_face_features = overlap_positive_features[:5]

df_kbof = pd.DataFrame(X_kBof, columns=features)
df_kbof['Cluster'] = kBof_clusters

# Select relevant features and cluster column
selected_columns = top10_negative_face_features + top10_positive_face_features + ['Cluster']
df_selected = df_kbof[selected_columns]

# Group by cluster and compute means
grouped = df_selected.groupby('Cluster')

# Compute sums of means for negative features
neg_feature_means = grouped[top10_negative_face_features].mean()
neg_sums = neg_feature_means.sum(axis=1)  # Sum of means, NOT mean of means
neg_means = grouped[top10_negative_face_features].mean().mean(axis=1)
pos_means = grouped[top10_positive_face_features].mean().mean(axis=1)

pos_feature_means = grouped[top10_positive_face_features].mean()
pos_sums = pos_feature_means.sum(axis=1)

# Compute and sort differences
difference = neg_means - pos_means
diff_df = pd.DataFrame({
    'Mean_Negative': neg_means,
    'Mean_Positive': pos_means,
    'Difference': difference,
    'Abs_Diff': difference.abs()
}).sort_values(by='Difference', ascending=False)

# Display
print("Cluster-wise differences between mean negative and positive face frequencies (N58 only):")
print(diff_df)

In [ ]:
#Choose the 10 clusters with highest difference in mean feature appearances to eliminate
highlight_clusters = diff_df.sort_values("Difference", ascending=False).head(10).index.tolist()

In [ ]:
min_val = table2['Pressure_slope'].min()
max_val = table2['Pressure_slope'].max()

# Avoid division by zero
if max_val == min_val:
    table2['Norm_Pressure'] = 0.5
else:
    table2['Norm_Pressure'] = (table2['Pressure_slope'] - min_val) / (max_val - min_val)

In [ ]:
#Retrieve over-valued anomalies from dataset (HCLP) based on defined threshold

top_10_cpm_threshold = table2['Count_Clean'].quantile(0.9)
bottom_10_enrichment_threshold = table2['Norm_Pressure'].quantile(0.1)

mask_top_cpm_bottom_enrichment = (table2['Count_Clean'] >= top_10_cpm_threshold) & \
                                  (table2['Norm_Pressure'] <= bottom_10_enrichment_threshold)

hclp = table2[mask_top_cpm_bottom_enrichment].index.tolist()

###t-SNE plot Figure 5

In [ ]:
from sklearn.manifold import TSNE

X_tsne = TSNE(n_components=2, perplexity=100, random_state=1).fit_transform(W)

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

# Compute t-SNE
X_tsne = TSNE(n_components=2, perplexity=100, random_state=1).fit_transform(W)

# Step 2: Map aptamer index → t-SNE coords (using table.index, not range)
tsne_index_to_coord = dict(zip(table.index, X_tsne))

# Step 3: Clustering results aligned to table.index
index_to_cluster = dict(zip(table.index, kBof_clusters))
unique_clusters = np.unique(kBof_clusters)
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_clusters)))

# Step 4: Group points by cluster
cluster_to_points = {}
for cluster in unique_clusters:
    idxs = [i for i, c in index_to_cluster.items() if c == cluster]
    cluster_to_points[cluster] = np.array([tsne_index_to_coord[i] for i in idxs])

# Step 5: Plot clusters
fig, ax = plt.subplots(figsize=(12, 6))
scatter_handles = []
for cluster, color in zip(unique_clusters, colors):
    points = cluster_to_points[cluster]
    sc = ax.scatter(points[:, 0], points[:, 1], s=80,
                    color=color, alpha=0.5,
                    label=f"Cluster {cluster}")
    scatter_handles.append(sc)

# Red crosses for all aptamers in hclp
for idx in hclp:
    if idx in tsne_index_to_coord:
        x, y = tsne_index_to_coord[idx]
        ax.scatter(x, y, marker='x', s=60, color='red',
                   linewidths=1.2, zorder=11, alpha=0.9)

# Thick green crosses (example)
green_cross_aptamers = [0, 35, 77, 87]
for apt in green_cross_aptamers:
    if apt in tsne_index_to_coord:
        x, y = tsne_index_to_coord[apt]
        ax.scatter(x, y, marker='x', s=120, color='lightgreen',
                   linewidths=2.5, zorder=12)

# Mark lowest-pressure aptamer
min_idx = table['Norm_Pressure'].idxmin()
if min_idx in tsne_index_to_coord:
    x, y = tsne_index_to_coord[min_idx]
    ax.scatter(x, y, s=100, c='red', marker='v',
               edgecolors='black', linewidths=1.5, zorder=10)

# --- Final formatting ---
ax.set_title("t-SNE of Aptamers (Spectral Clusters)")
ax.set_xlabel("t-SNE 1")
ax.set_ylabel("t-SNE 2")
ax.axis('off')
ax.legend(handles=scatter_handles, title="Cluster",
          loc='upper right', bbox_to_anchor=(1.25, 1.0), fontsize=8)

plt.tight_layout()
plt.show()


## Hypothesis generation

In [ ]:
#Positive features in general (overlap or not)
top_pos_features = positive_coef_df['Feature'].tolist()
feature_to_index = {name: idx for idx, name in enumerate(features)}
valid_pos_features = [name for name in top_pos_features if name in feature_to_index]

# Index and extract columns corresponding to positive indices
pos_feature_indices = [feature_to_index[name] for name in valid_pos_features]
C_prime = X_kBof[:, pos_feature_indices]

In [ ]:
# Create mask for non-highlighted aptamers
non_highlight_mask = ~np.isin(kBof_clusters, list(highlight_clusters))

# Step 6: Subset C_prime using this mask
C_nonhighlight = C_prime[non_highlight_mask]

In [ ]:
# Ensure all values in C are non-negative
# Use MinMaxScaler to scale each feature between 0 and 1
scaler_nmf = MinMaxScaler()
C_nonneg = scaler_nmf.fit_transform(C_nonhighlight)

# Fit NMF on matrix of remaining aptamers and positive features
nmf_model = NMF(n_components=25, init='nndsvda', random_state=42, max_iter=1000)
W = nmf_model.fit_transform(C_nonneg)  # Aptamer-topic matrix (samples × components)
H = nmf_model.components_              # Feature-topic matrix (components × features)

# Run Spectral Clustering
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
kBof_clusters_nonhighlight = spectral.fit_predict(W)

In [ ]:
# Step 1: Convert to NumPy arrays
kBof_clusters = np.array(kBof_clusters)
table_indices = np.arange(len(table2))  # original indices (global row numbers)

# Step 2: Boolean mask for non-highlighted clusters
non_highlight_mask = ~np.isin(kBof_clusters, list(highlight_clusters))

# Step 3: Apply mask
unhighlighted_orig_indices = table_indices[non_highlight_mask]          # Global indices in table2
unhighlighted_local_indices = np.arange(len(table2))[non_highlight_mask]  # Their positions within the unhighlighted subset

# Immediately reindex the cluster labels to 0–14
unique_labels = np.unique(kBof_clusters_nonhighlight)
label_remap = {old: new for new, old in enumerate(sorted(unique_labels))}
kBof_clusters_nonhighlight = np.array([label_remap[cl] for cl in kBof_clusters_nonhighlight])


In [ ]:
# Step 1: Combine top features and map to indices
combined_top_features = top10_negative_face_features + top10_positive_face_features
feature_to_index = {name: idx for idx, name in enumerate(features)}
combined_feature_indices = [feature_to_index[name] for name in combined_top_features]
subset_features = [features[i] for i in combined_feature_indices]

# Step 2: Get feature matrix for **non-highlighted aptamers only**
C_combined_nonhighlight = X_kBof[non_highlight_mask][:, combined_feature_indices]

# Step 4: Build DataFrame
df_kbof_new = pd.DataFrame(C_combined_nonhighlight, columns=subset_features)
df_kbof_new['Cluster'] = kBof_clusters_nonhighlight  # shape matches number of rows

# Step 5: Select relevant columns and group
selected_columns = top10_negative_face_features + top10_positive_face_features + ['Cluster']
df_selected_new = df_kbof_new[selected_columns]
grouped_new = df_selected_new.groupby('Cluster')

# Step 6: Compute mean face scores and difference
neg_means_new = grouped_new[top10_negative_face_features].mean().mean(axis=1)
pos_means_new = grouped_new[top10_positive_face_features].mean().mean(axis=1)
difference_new = pos_means_new - neg_means_new

# Step 7: Build final DataFrame
diff_df_nonhighlight = pd.DataFrame({
    'Mean_Positive': pos_means_new,
    'Mean_Negative': neg_means_new,
    'Difference': difference_new
}).sort_values(by='Difference', ascending=False)

# Step 8: (Optional) Display
print("Cluster-wise (Positive - Negative) face frequency means (Non-highlighted aptamers only):")
print(diff_df_nonhighlight)

In [ ]:
# Extract clusters with the highest 10 positive differences
clusters_with_positive_diff = (
    diff_df_nonhighlight.head(10).index.tolist()
)

In [ ]:
# Get the indices of aptamers used in W (non-highlighted)
W_index = table.index[non_highlight_mask].tolist()

In [ ]:
# mapping aptamer index (from W_index) to new cluster label
index_to_cluster_2 = dict(zip(W_index, kBof_clusters_nonhighlight))

# mapping new cluster label to list of aptamer indices
cluster_to_indices_2 = defaultdict(list)
for idx in W_index:
    cluster = index_to_cluster_2[idx]
    cluster_to_indices_2[cluster].append(idx)

###t-SNE plot figure 6

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

# t-SNE on W (from NMF on remaining aptamers, positive features)
tsne = TSNE(n_components=2, perplexity=100, random_state=1)
X_tsne = tsne.fit_transform(W)

# Map aptamer index to t-SNE coordinates
tsne_index_to_coord = dict(zip(W_index, X_tsne))

# Group t-SNE points by new clusters
fig, ax = plt.subplots(figsize=(12, 6))
unique_clusters = np.unique(list(index_to_cluster_2.values()))
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_clusters)))

cluster_to_points = defaultdict(list)
cluster_to_indices = defaultdict(list)
for idx in W_index:
    cluster = index_to_cluster_2[idx]
    cluster_to_points[cluster].append(tsne_index_to_coord[idx])
    cluster_to_indices[cluster].append(idx)

# Manually override colors for specific clusters
custom_colors = {}
for i, cluster in enumerate(unique_clusters):
    if cluster in {12}:
        custom_colors[cluster] = 'pink'    # override green with pink
    elif cluster in {5}:
        custom_colors[cluster] = 'lightpink'
    else:
        custom_colors[cluster] = colors[i]

# Plot each cluster
scatter_handles = []
for cluster in unique_clusters:
    points = np.array(cluster_to_points[cluster])
    sc = ax.scatter(points[:, 0], points[:, 1], s=80,
                    color=custom_colors[cluster], alpha=0.5,
                    label=f"Cluster {cluster}")
    scatter_handles.append(sc)


# Mark selected aptamers with lightgreen contour circle
green_cross_aptamers = [0, 35, 77, 87]
for apt in green_cross_aptamers:
    if apt in tsne_index_to_coord:
        x, y = tsne_index_to_coord[apt]
        ax.scatter(x, y, marker='x', s=120, color='green', linewidths=1, zorder=14)

# Mark global max pressure and max count with blue triangle
max_idx = table['Norm_Pressure'].idxmax()
if max_idx in tsne_index_to_coord:
    x, y = tsne_index_to_coord[max_idx]
    ax.scatter(x, y, s=100, c='blue', marker='^',
               edgecolors='black', linewidths=1.5, zorder=10)

aptamer_0 = 0
if aptamer_0 in tsne_index_to_coord:
    x, y = tsne_index_to_coord[aptamer_0]
    ax.scatter(x, y, s=100, c='blue', marker='^',
               edgecolors='black', linewidths=1.5, zorder=10)

# Cyan contour circles for max count / pressure in positive-diff clusters
all_marked = []
for cluster in clusters_with_positive_diff:
    if cluster not in cluster_to_indices:
        print(f"Cluster {cluster} not found. Skipping.")
        continue

    full_cluster_indices = cluster_to_indices[cluster]
    cluster_df = table.loc[full_cluster_indices]

    max_count_idx = cluster_df['Count_Clean'].idxmax()
    max_press_idx = cluster_df['Norm_Pressure'].idxmax()

    all_marked.extend([max_count_idx, max_press_idx])

# Plot all 20 aptamers with cyan circle (if present in t-SNE)
for idx in set(all_marked):
    if idx in tsne_index_to_coord:
        x, y = tsne_index_to_coord[idx]
        ax.scatter(x, y, marker='x', s=120, color='darkblue', linewidths=1.5, zorder=14)

# Final formatting
ax.set_title("t-SNE of Aptamers (W from Positive Features Only)")
ax.set_xlabel("t-SNE 1")
ax.set_ylabel("t-SNE 2")
ax.axis('off')
ax.grid(False)
ax.legend(handles=scatter_handles, title="Cluster",
          loc='upper right', bbox_to_anchor=(1.25, 1.0), fontsize=8)

plt.tight_layout()
plt.show()